# Programación lineal

## 1. Problemas de programación lineal

De acuerdo a lo visto la clase pasada, un problema de programación lineal puede escribirse en la siguiente forma:

\begin{equation}
\begin{array}{ll}
\min_{x_1,\dots,x_n} & f_1x_1+\dots+f_nx_n \\
\text{s. a. }        & a^{eq}_{j,1}x_1+\dots+a^{eq}_{j,n}x_n=b^{eq}_j \text{ para } 1\leq j\leq m_1 \\
                     & a_{k,1}x_1+\dots+a_{k,n}x_n\leq b_k \text{ para } 1\leq k\leq m_2,
\end{array}
\end{equation}
donde:
- $x_i$ para $i=1,\dots,n$ son las incógnitas o variables de decisión,
- $f_i$ para $i=1,\dots,n$ son los coeficientes de la función a optimizar,
- $a^{eq}_{j,i}$ para $j=1,\dots,m_1$ e $i=1,\dots,n$, son los coeficientes de la restricción de igualdad,
- $a_{k,i}$ para $k=1,\dots,m_2$ e $i=1,\dots,n$, son los coeficientes de la restricción de desigualdad,
- $b^{eq}_j$ para $j=1,\dots,m_1$ son valores conocidos que deben ser respetados estrictamente, y
- $b_k$ para $k=1,\dots,m_2$ son valores conocidos que no deben ser superados.

Equivalentemente, el problema puede escribirse como

\begin{equation}
\begin{array}{ll}
\min_{\boldsymbol{x}} & \boldsymbol{f}^T\boldsymbol{x} \\
\text{s. a. }        & \boldsymbol{A}_{eq}\boldsymbol{x}=\boldsymbol{b}_{eq} \\
                     & \boldsymbol{A}\boldsymbol{x}\leq\boldsymbol{b},
\end{array}
\end{equation}
donde:
- $\boldsymbol{x}=\left[x_1\quad\dots\quad x_n\right]^T$,
- $\boldsymbol{f}=\left[f_1\quad\dots\quad f_n\right]^T$,
- $\boldsymbol{A}_{eq}=\left[\begin{array}{ccc}a^{eq}_{1,1} & \dots & a^{eq}_{1,n}\\ \vdots & \ddots & \vdots\\ a^{eq}_{m_1,1} & \dots & a^{eq}_{m_1,n}\end{array}\right]$,
- $\boldsymbol{A}=\left[\begin{array}{ccc}a_{1,1} & \dots & a_{1,n}\\ \vdots & \ddots & \vdots\\ a_{m_2,1} & \dots & a_{m_2,n}\end{array}\right]$,
- $\boldsymbol{b}_{eq}=\left[b^{eq}_1\quad\dots\quad b^{eq}_{m_1}\right]^T$, y
- $\boldsymbol{b}=\left[b_1\quad\dots\quad b_{m_2}\right]^T$.

**Nota:** el problema $\max_{\boldsymbol{x}}\boldsymbol{g}(\boldsymbol{x})$ es equivalente a $\min_{\boldsymbol{x}}-\boldsymbol{g}(\boldsymbol{x})$.

## 2. Explicar la función pyomo_utilities.py

Para trabajar con `pyomo`:
1. `pyomo` a parte de contener funciones para optimización, es en sí un lenguaje de programación. No tiene solucionadores instalados, utiliza solucionadores externos (glpk, ipopt).
2. Modelos concretos y modelos abstractos.
3. Forma de ingresarle los parámetros para un modelo abstracto.
4. Resultados.

## 3. Ejemplos de la clase pasada

### 3.1 
Una compañía produce dos productos ($X_1$ y $X_2$) usando dos máquinas ($A$ y $B$). Cada unidad de $X_1$ que se produce requiere 50 minutos en la máquina $A$ y 30 minutos en la máquina $B$. Cada unidad de $X_2$ que se produce requiere 24 minutos en la máquina $A$ y 33 minutos en la máquina $B$.

Al comienzo de la semana hay 30 unidades de $X_1$ y 90 unidades de $X_2$ en inventario. El tiempo de uso disponible de la máquina $A$ es de 40 horas y el de la máquina $B$ es de 35 horas.

La demanda para $X_1$ en la semana actual es de 75 unidades y de $X_2$ es de 95 unidades. La política de la compañía es maximizar la suma combinada de unidades de $X_1$ e $X_2$ en inventario al finalizar la semana.

Formular el problema de decidir cuánto hacer de cada producto en la semana como un problema de programación lineal.

### Solución

Sean:
- $x_1$ la cantidad de unidades de $X_1$ a ser producidas en la semana, y
- $x_2$ la cantidad de unidades de $X_2$ a ser producidas en la semana.

\begin{equation}
\begin{array}{ll}
\min_{\boldsymbol{x}} & \boldsymbol{f}^T\boldsymbol{x} \\
\text{s. a. }        & \boldsymbol{A}_{eq}\boldsymbol{x}=\boldsymbol{b}_{eq} \\
                     & \boldsymbol{A}\boldsymbol{x}\leq\boldsymbol{b},
\end{array}
\end{equation}
con
- $\boldsymbol{f}=\left[-1 \quad -1\right]^T$,
- $\boldsymbol{A}_{eq}=\left[0\quad 0\right]$,
- $\boldsymbol{A}=\left[\begin{array}{cc}50 & 24 \\ 30 & 33\\ -1 & 0\\ 0 & -1\end{array}\right]$,
- $\boldsymbol{b}_{eq}=0$, y
- $\boldsymbol{b}=\left[2400\quad 2100\quad -45\quad -5\right]^T$.

In [1]:
!cat pyomo_utilities.py

"""
Author: Esteban Jiménez Rodríguez
Institution: ITESO - Universidad Jesuita de Guadalajara
Date: 01/11/2017
Description: This script contains functions that make possible to use the pyomo
library directly.
"""

from __future__ import division
import numpy as np
import pyomo.environ
import pyomo.opt
    
def dat_write_lin(f, A, b, Aeq, beq):
    # Dimensions of matrices
    m1 = b.shape[0]
    n = f.shape[0]
    m2 = beq.shape[0]

    # Creation and opening of the data file
    dat_file = open('default.dat', 'w')
    
    # Write the matrices in the data file
    dat_file.write('# This file is automatically generated by the linprog function\n');
    dat_file.write('param m1 := %i ;\nparam m2 := %i ;\nparam n := %i ;\nparam A := \n'%(m1, m2, n));
    for i in range(m1):
        for k in range(n):
            dat_file.write('%i %i %e\n'%(i+1, k+1, A[i, k]));
    dat_file.write(';\nparam b := \n');
    for i in range(m1):
        dat_file.write('%i %e\n'%(i+1, b[i]));
    dat_file.write

In [2]:
import numpy as np
f = np.array([2, 3])
A = np.array([[-3, -4]])
b = np.array([1])
Aeq = np.array([[0, 0]])
beq = np.array([0])

In [3]:
import pyomo_utilities
pyomo_utilities.dat_write_lin(f, A, b, Aeq, beq)

### 3.2
Mónica hace aretes y cadenitas de joyería. Es tan buena, que todo lo que hace lo vende.

Le toma 30 minutos hacer un par de aretes y una hora hacer una cadenita, y como Mónica también es estudihambre, solo dispone de 10 horas a la semana para hacer las joyas. Por otra parte, el material que compra solo le alcanza para hacer 15 unidades (el par de aretes cuenta como unidad) de joyas por semana.

La utilidad que le deja la venta de las joyas es \$15 en cada par de aretes y \$20 en cada cadenita.

¿Cuántos pares de aretes y cuántas cadenitas debería hacer Mónica para maximizar su utilidad?

Formular el problema en la forma explicada y obtener la solución gráfica (puede ser a mano).

### Solución

Sean:
- $x_1$ la cantidad de pares de aretes que hace Mónica.
- $x_2$ la cantidad de cadenitas que hace Mónica.


\begin{equation}
\begin{array}{ll}
\min_{\boldsymbol{x}} & \boldsymbol{f}^T\boldsymbol{x} \\
\text{s. a. }        & \boldsymbol{A}_{eq}\boldsymbol{x}=\boldsymbol{b}_{eq} \\
                     & \boldsymbol{A}\boldsymbol{x}\leq\boldsymbol{b},
\end{array}
\end{equation}
con
- $\boldsymbol{f}=\left[-15 \quad -20\right]^T$,
- $\boldsymbol{A}_{eq}=\left[0\quad 0\right]$,
- $\boldsymbol{A}=\left[\begin{array}{cc}0.5 & 1 \\ 1 & 1\\ -1 & 0\\ 0 & -1\end{array}\right]$,
- $\boldsymbol{b}_{eq}=0$, y
- $\boldsymbol{b}=\left[10\quad 15\quad 0\quad 0\right]^T$.

## 4. Problema de transporte

- **Referencia**: https://es.wikipedia.org/wiki/Programaci%C3%B3n_lineal

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/a/a0/Progr_Lineal.PNG" width="400px" height="125px" />

Este es un caso curioso, con solo 6 variables (un caso real de problema de transporte puede tener fácilmente más de 1.000 variables) en el cual se aprecia la utilidad de este procedimiento de cálculo.

Existen tres minas de carbón cuya producción diaria es:
- la mina "a" produce 40 toneladas de carbón por día;
- la mina "b" produce 40 t/día; y,
- la mina "c" produce 20 t/día.

En la zona hay dos centrales termoeléctricas que consumen:
- la central "d" consume 40 t/día de carbón; y,
- la central "e" consume 60 t/día.

Los costos de mercado, de transporte por tonelada son:
- de "a" a "d" = 2 monedas;
- de "a" a "e" = 11 monedas;
- de "b" a "d" = 12 monedas;
- de "b" a "e" = 24 monedas;
- de "c" a "d" = 13 monedas; y,
- de "c" a "e" = 18 monedas.

Si se preguntase a los pobladores de la zona cómo organizar el transporte, tal vez la mayoría opinaría que debe aprovecharse el precio ofrecido por el transportista que va de "a" a "d", porque es más conveniente que los otros, debido a que es el de más bajo precio.

En este caso, el costo total del transporte es:
- transporte de 40 t de "a" a "d" = 80 monedas;
- transporte de 20 t de "c" a "e" = 360 monedas; y,
- transporte de 40 t de "b" a "e" = 960 monedas,
    
Para un total 1.400 monedas.

Sin embargo, formulando el problema para ser resuelto por la programación lineal se tienen las siguientes ecuaciones:

Restricciones de la producción:

- $x_1 + x_2 \leq 40$
- $x_3 + x_4 \leq 40$
- $x_5 + x_6 \leq 20$

Restricciones del consumo:

- $x_1 + x_3 + x_5 \geq 40$
- $x_2 + x_4 + x_6 \geq 60$

La función objetivo será:

$$\min_{x_1,\dots,x_6}2x_1 + 11x_2 + 12x_3 + 24x_4 + 13x_5 + 18x_6$$

La solución de costo mínimo de transporte diario resulta ser:

- $x_2 = 40$ resultando un costo de $11(40) = 480$ monedas
- $x_3 = 40$ resultando un costo de $12(40) = 440$ monedas
- $x_6 = 20$ resultando un costo de $18(20) = 360$ monedas
    
para un total de $1280$ monedas, $120$ monedas menos que antes.

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>